In [ ]:
!pip install wandb

Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import wandb
from wandb.keras import WandbCallback

Init Wandb

In [ ]:
wandb.init(project="CNN - Mnist")

accuracy,▁▆▇▇▇▇█████████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▂▄▆▄█▆▅▆▇▆▆▄▆
val_loss,▅▄▅▃▂▄▁▆▂▅▁▄▄█▆
accuracy,0.99846
best_epoch,6
best_val_loss,0.04325
epoch,14
loss,0.00575
val_accuracy,0.98717


In [ ]:
config = wandb.config
config.learning_rate = 0.001

Dataset

In [ ]:
mnist = tf.keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=33)

X_train, X_test, X_val = X_train / 255.0, X_test / 255.0, X_val / 255.0
print(X_train.shape, X_val.shape, X_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(54000, 28, 28) (6000, 28, 28) (10000, 28, 28)


Model

In [ ]:
model = tf.keras.models.Sequential([
    # features extraction
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (5, 5), activation="relu"),

    layers.Flatten(),

    # Network Layers
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

Compile

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=["accuracy"]
)

Summary

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 64)          102464    
                                                                 
 flatten (Flatten)           (None, 64)                0

Train

In [ ]:
model.fit(X_train, Y_train, epochs=15, validation_data=(X_val, Y_val), callbacks=[WandbCallback()])

Epoch 1/15
1688/1688 [==============================] - 22s 7ms/step - loss: 0.1640 - accuracy: 0.9498 - val_loss: 0.0655 - val_accuracy: 0.9785 - _timestamp: 1646546916.0000 - _runtime: 914.0000
Epoch 2/15
1688/1688 [==============================] - 13s 7ms/step - loss: 0.0519 - accuracy: 0.9831 - val_loss: 0.0612 - val_accuracy: 0.9812 - _timestamp: 1646546929.0000 - _runtime: 927.0000
Epoch 3/15
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0353 - accuracy: 0.9896 - val_loss: 0.0689 - val_accuracy: 0.9795 - _timestamp: 1646546941.0000 - _runtime: 939.0000
Epoch 4/15
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0269 - accuracy: 0.9914 - val_loss: 0.0547 - val_accuracy: 0.9828 - _timestamp: 1646546954.0000 - _runtime: 952.0000
Epoch 5/15
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0208 - accuracy: 0.9934 - val_loss: 0.0509 - val_accuracy: 0.9863 - _timestamp: 1646546967.0000 - _runtime: 965.0000
Epoch 6/15
1688/1688

In [ ]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0512 - accuracy: 0.9901


[0.05122917518019676, 0.9901000261306763]

Inference

In [ ]:
import cv2

img = cv2.imread("7_mnist.png")
img = cv2.resize(img, (28, 28))
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img = img / 255.0
img = img.reshape(1, 28, 28)

result = model.predict(img)
np.argmax(result)
# result

7